In [1]:
import pandas as pd
import numpy as np
import os
import math

import librosa

from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import matplotlib.pyplot as plt

In [2]:
# path_to_file = 'C:/Users/USER/Downloads/Tugas Akhir/dataset/tes/'
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/tes/fitur/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"

path_to_file = 'F:/178/Tugas Akhir/dataset/tes/'
path_to_fitur = "F:/178/Tugas Akhir/dataset/tes/fitur/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"

meanstd = pd.read_csv(path_to_experiment+"case1_meanstd.csv",header=None)
meanstd = meanstd.values

In [17]:
# #Load
def load(src):
    y, sr = librosa.load(src,mono=True,sr=22050)
    
    return y,sr

# #Convert
def convert(filename,path,y,sr):
    #Path
    dst = path+filename[:-4]+".wav"
    
    #COnver to WAV
    librosa.output.write_wav(dst, y, sr)

# #Get Feature

# #MFCC
def mfcc(y,sr,file):
    vector = list()
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13) #13 is default dimension 512 frame
    data = pd.DataFrame(mfcc)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,13,std))
    save = pd.DataFrame(vector)

    save.to_csv(path_to_fitur+file[:-4]+"/mfcc.csv", index=False, header=False)

# #TIMBRE // CENTROID // FLUX // ROLLOFF // ZERO CROSSING 
def timbre(y,sr,file):
    vector = list()
    cent = librosa.feature.spectral_centroid(y=y, sr=sr) #1 is default dimension 512 frame
    flux = librosa.onset.onset_strength(y=y, sr=sr) #1 is default dimension 512 frame
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr) #1 is default dimension 512 frame
    zc = librosa.feature.zero_crossing_rate(y) #1 is default dimension 512 frame
    
    a = pd.DataFrame(cent) 
    b = pd.DataFrame(flux).T
    c = pd.DataFrame(rolloff)
    d = pd.DataFrame(zc)
    
    frame = [a,b,c,d]
    data = pd.concat(frame)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,4,std))
    save = pd.DataFrame(vector)
    
    save.to_csv(path_to_fitur+file[:-4]+"/timbre.csv", index=False, header=False)

# #SCF and SFM
def flatness(y,sr,file):
    vector = list()
    flatness = librosa.feature.spectral_flatness(y=y)
    data = pd.DataFrame(flatness)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,1,std))
    save = pd.DataFrame(vector)
    
    save.to_csv(path_to_fitur+file[:-4]+"/flatness.csv", index=False, header=False)
    
def crest(y,sr,file):
    peak = y.max()
    rms = librosa.feature.rmse(y=y)
    n = rms.size
    square = rms**2
    rms_v2 = math.sqrt((1/n)*(square.sum()))
    crest = [peak/rms_v2]
    
    save = pd.DataFrame(crest)
    
    save.to_csv(path_to_fitur+file[:-4]+"/crest.csv", index=False, header=False)

# #chroma
def chroma(y,sr,file):
    vector = list()
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_chroma=12) #24 is default dimension 512 frame
    data = pd.DataFrame(chroma)
    mean = data.mean(axis = 1).values #Get Mean
    std = data.std(axis = 1).values #Get std
    vector.append(np.insert(mean,12,std))
    save = pd.DataFrame(vector)
    
    save.to_csv(path_to_fitur+file[:-4]+"/chroma.csv", index=False, header=False)

def fitur(file,y,sr):
    
    mfccs = mfcc(y,sr,file) #mfcc
    timbres = timbre(y,sr,file) #timbre
    flatnesss = flatness(y,sr,file) #sfm
    crests = crest(y,sr,file) #scf
    chromas = chroma(y,sr,file) #chroma
    
# #merge feature

def merge(case1,case2):
    case_a = pd.DataFrame(case1)
    case_b = pd.DataFrame(case2)
    
    merged = case_a.merge(case_b, how='outer', left_index=True, right_index=True)
    
    return merged

def flat(song,fitur):
    vector = list()
    
    data = pd.read_csv(path_to_fitur+str(song)+"/"+fitur+".csv",header=None)
    vector.append(np.squeeze(data.values,0))

    return vector

def norm(data):
    result = np.array(data)
    mean = meanstd[0]
    std = meanstd[1]
    normalized = (result-mean)/std

def case1(file):
    fitur = 'mfcc'
    result = flat(file,fitur)
    
    return result

def case2(file):
    fitur = 'timbre'
    result = flat(file,fitur)
    result = np.array(result)
    mean = meanstd[1][0]
    std = meanstd[1][1]
    normalized = (result-mean)/std
    
    return normalized

def case3(file):
    fitur1 = 'crest'
    fitur2 = 'flatness'
    crest = flat(file,fitur1)
    flatness = flat(file,fitur2)
    result = merge(crest,flatness)
    result = np.array(result)
    mean = meanstd[2][0]
    std = meanstd[2][1]
    normalized = (result-mean)/std
    
    return normalized
#     result = norm(result)
    
#     return result

def case4(file):
    fitur = 'chroma'
    result = flat(file,fitur)
    result = np.array(result)
    mean = meanstd[3][0]
    std = meanstd[3][1]
    normalized = (result-mean)/std
    
    return normalized
#     result = pd.DataFrame(data=result)
#     result = norm(result)
    
#     return result

def case5(file):
    case_1 = case1(file)
    case_2 = case2(file)
    result = merge(case_1,case_2)
    
    return result

def case6(file):
    case_1 = case1(file)
    case_2 = case3(file)
    result = merge(case_1,case_2)
    
    return result

def case7(file):
    case_1 = case1(file)
    case_2 = case4(file)
    result = merge(case_1,case_2)
    
    return result

def case8(file):
    case_1 = case5(file)
    case_2 = case3(file)
    result = merge(case_1,case_2)
    
    return result

def case9(file):
    case_1 = case5(file)
    case_2 = case4(file)
    result = merge(case_1,case_2)
    
    return result

def case10(file):
    case_1 = case8(file)
    case_2 = case4(file)
    result = merge(case_1,case_2)
    
    return result


# #Get emotion

def emotion(a,v):
    if ((v > 0) & (a >= 0)):
        cluster = 'cluster 1'
    if ((v <= 0) & (a > 0)):
        cluster = 'cluster 2'
    if ((v < 0) & (a <= 0)):
        cluster = 'cluster 3'
    if ((v >= 0) & (a < 0)):
        cluster = 'cluster 4'
        
    return cluster

def plot_scatter(a,v):
    plt.figure(figsize=(15, 15))
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    bbox=dict(boxstyle="round",
              ec=(1., 0.5, 0.5),
              fc=(1., 0.8, 0.8),
              alpha=0.6,
              color = 'yellow'
           )
    plt.text(0.6, 0.7, "cluster 1", size=30,
         ha="center", va="center",
         bbox=bbox
         )
    plt.text(-0.6, 0.7, "cluster 2", size=30,
         ha="center", va="center",
         bbox=bbox
         )
    plt.text(-0.6, -0.7, "cluster 3", size=30,
         ha="center", va="center",
         bbox=bbox
         )
    plt.text(0.6, -0.7, "cluster 4", size=30,
         ha="center", va="center",
         bbox=bbox
         )

    plt.scatter(v,a,color='b')
    plt.axhline(y=0,color='black', label ="X")
    plt.axvline(x=0,color='black', label ="Y")
    plt.show()
    
def proses(path):
    #Split path
    drive, path_and_file = os.path.splitdrive(path)
    paths, file = os.path.split(path_and_file)
    path_wav = drive+paths+'\wav\\'
    path_fitur = drive+paths+'\fitur\\'
    #load raw
    y, sr = load(path)

    #Convert Wav
    convert(file,path_wav,y,sr)

    #Load Wav
    y_wav, sr_wav = load(path_wav+file[:-4]+'.wav')

    #make dir
    try:  
        os.mkdir(path_fitur+file[:-4])
    except OSError:  
        print ("Creation of the directory %s failed" % path_fitur)

    # #Feature extraction
    fiture = fitur(file[:-4]+'.wav',y_wav,sr_wav)
    print("feature successfully")

    # #Flatten
    results = case1(file[:-4])
    print("flatten successfully")

    # #Norm
    results = norm(results)
    
    # #Load Model

    arousal = path_to_model+'arousal/case1.sav'
    valence = path_to_model+'valence/case1.sav'

    arousal_model = joblib.load(arousal)
    valence_model = joblib.load(valence)

    a = arousal_model.predict(results)
    v = valence_model.predict(results)
    
    emo = emotion(a,v)
    
#     return v,a,normalized
    return a,v,emo,results

#     emo = emotion(a,v)

In [18]:
a,v,emo,normalized = proses("F:\\178\\Tugas Akhir\\dataset\\tes\\939.mp3")
print(a.round(2),v.round(2),emo)

Creation of the directory F:\178\Tugas Akhir\dataset\tesitur\ failed
feature successfully
flatten successfully


ValueError: Expected 2D array, got scalar array instead:
array=None.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [74]:
#1 [ 0.08][ 0.21] 1 
#2 [-0.13][ 0.03] 4 
#3 [ 0.01][ 0.08] 1 
#4 [-0.37][-0.12] 3 [-0.02][ 0.15]  4
#5 [ 0.06][ 0.17] 1 [ 0.36][ 0.23] 1
#6 [ 0.07][ 0.24] 1 [ 0.34][ 0.24] 1
#7 [-0.32] [-0.67] cluster 3
#8 [-0.25] [-0.52] cluster 3
#9 [-0.28] [-0.49] cluster 3 [-0.3] [-0.44] cluster 3
#10 [-0.28] [-0.44] cluster 3 [-0.3] [-0.38] cluster 3


#Load Model

arousal = path_to_model+'arousal/case1.sav'
valence = path_to_model+'valence/case1.sav'

arousal_model = joblib.load(arousal)
valence_model = joblib.load(valence)
arousal_model.get_params()
# a = arousal_model.predict(result)
# v = valence_model.predict(result)

# emo = emotion(a,v)

{'alpha': 1e-10,
 'copy_X_train': True,
 'kernel': 1.17**2 * RationalQuadratic(alpha=0.022, length_scale=4.78),
 'kernel__k1': 1.17**2,
 'kernel__k1__constant_value': 1.362172982755161,
 'kernel__k1__constant_value_bounds': (1e-05, 100000.0),
 'kernel__k2': RationalQuadratic(alpha=0.022, length_scale=4.78),
 'kernel__k2__alpha': 0.02203730709759159,
 'kernel__k2__alpha_bounds': (1e-06, 1000000.0),
 'kernel__k2__length_scale': 4.7836885154724067,
 'kernel__k2__length_scale_bounds': (1e-06, 1000000.0),
 'n_restarts_optimizer': 10,
 'normalize_y': True,
 'optimizer': 'fmin_l_bfgs_b',
 'random_state': 1000}

In [ ]:
# for filename in os.listdir(path_to_file):
#     if os.path.isfile(os.path.join(path_to_file, filename)):
        
#         #load raw
#         y, sr = load(path_to_file+filename)

#         #Convert Wav
#         convert(filename,path_dst,y,sr)
        
#         #Load Wav
#         y_wav, sr_wav = load(path_to_file+'wav/'+file)

#         #make dir
#         try:  
#             os.mkdir(path_to_fitur+filename[:-4])
#         except OSError:  
#             print ("Creation of the directory %s failed" % path_to_fitur)
        
#         # #Feature extraction
#         fiture = fitur(filename[:-4]+'.wav',y_wav,sr_wav)
#         print("feature successfully")
        
#         # #Flatten
#         results = case4(filename[:-4])
#         print("flatten successfully")
        
#         # #Norm
#         normalized = norm(results)